In [1]:
import numpy as np
import pretty_midi
from tensorflow.keras.models import load_model


In [2]:
# Load the trained model
model = load_model('rnn_music_best_model.keras')

# Confirm model summary
model.summary()


2024-12-27 14:04:44.746842: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-12-27 14:04:44.747159: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-27 14:04:44.747269: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-27 14:04:44.747327: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-27 14:04:44.747365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 50, 256)        │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50, 512)        │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,613,314 (44.30 MB)

 Trainable params: 3,871,104 (14.77 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,742,210 (29.53 MB)

In [3]:
def generate_music(model, start_sequence, seq_length=100, output_length=500, temperature=1.0):
    """
    Generate a sequence of MIDI notes using the trained model.
    Args:
        model (tf.keras.Model): Trained RNN model.
        start_sequence (list): Initial sequence of notes to seed the generation.
        seq_length (int): Length of the input sequence.
        output_length (int): Desired length of the generated sequence.
        temperature (float): Sampling temperature to control randomness.
    Returns:
        list: Generated sequence of MIDI notes.
    """
    generated = list(start_sequence)  # Start with the seed sequence
    for _ in range(output_length):
        # Prepare input for the model
        input_seq = np.expand_dims(generated[-seq_length:], axis=0)  # Shape: (1, seq_length)
        
        # Predict probabilities for the next note
        predictions = model.predict(input_seq, verbose=0)[0]
        
        # Apply temperature for more diverse generation
        predictions = np.log(predictions + 1e-8) / temperature
        predictions = np.exp(predictions) / np.sum(np.exp(predictions))
        
        # Sample the next note
        next_note = np.random.choice(range(len(predictions)), p=predictions)
        generated.append(next_note)
    
    return generated


In [4]:
# Define a seed sequence (use part of your validation data or a custom sequence)
start_sequence = X_val[0][:seq_length]  # Use the first sequence from validation data

# Generate a new sequence
generated_sequence = generate_music(model, start_sequence, seq_length=100, output_length=500, temperature=1.0)

# Display the generated sequence
print("Generated sequence:", generated_sequence[:20])  # Print the first 20 notes


NameError: name 'X_val' is not defined